## Imports

In [8]:
import sqlite3
import pandas as pd

## Download data

In [9]:
# !curl awing.kis.agh.edu.pl:8080/detectors_names_traffic_s_small.csv.bz2 | bunzip2 > data.csv

In [10]:
db_connection = sqlite3.connect("proj6_readings.sqlite")
cur = db_connection.cursor()
pd.read_sql("SELECT * FROM readings LIMIT 10;", db_connection)

,detector_id,shortname,name,starttime,endtime,count
0,1,CAUTC11FD101_D1,\N,2015-06-22 23:58:31,2015-06-23 00:00:01,1
1,1,CAUTC11FD101_D1,\N,2015-06-23 00:00:01,2015-06-23 00:01:31,1
2,1,CAUTC11FD101_D1,\N,2015-06-23 00:01:31,2015-06-23 00:03:01,0
3,1,CAUTC11FD101_D1,\N,2015-06-23 00:03:01,2015-06-23 00:04:31,3
4,1,CAUTC11FD101_D1,\N,2015-06-23 00:04:31,2015-06-23 00:06:01,0
5,1,CAUTC11FD101_D1,\N,2015-06-23 00:06:01,2015-06-23 00:07:31,1
6,1,CAUTC11FD101_D1,\N,2015-06-23 00:07:31,2015-06-23 00:09:01,0
7,1,CAUTC11FD101_D1,\N,2015-06-23 00:09:01,2015-06-23 00:10:31,1
8,1,CAUTC11FD101_D1,\N,2015-06-23 00:10:31,2015-06-23 00:12:01,1
9,1,CAUTC11FD101_D1,\N,2015-06-23 00:12:01,2015-06-23 00:13:31,1


## Add indexes (optimization)

In [11]:
cur.execute("""
CREATE INDEX IF NOT EXISTS detector_id ON readings (detector_id);
""").fetchall()
cur.execute("""
CREATE INDEX IF NOT EXISTS starttime ON readings (starttime);
""").fetchall()

[]

## 2.1 Counting

In [12]:
df = pd.read_sql("SELECT COUNT(DISTINCT detector_id) FROM readings;", db_connection)
print(df)

# save to pickle
df.to_pickle("e proj6_ex01_detector_no.pkl")

   COUNT(DISTINCT detector_id)
0                          146


## 2.2 Some stats for the detectors


In [13]:
df = pd.read_sql("""
SELECT detector_id, COUNT(*) AS measurement_count, MIN(starttime) AS min_starttime, MAX(starttime) AS max_starttime
FROM readings
WHERE count IS NOT NULL
GROUP BY detector_id;
""", db_connection)
print(df)

# save to pickle
df.to_pickle("proj6_ex02_detector_stat.pkl")


     detector_id  measurement_count        min_starttime        max_starttime
0              1             357214  2015-06-22 23:58:31  2016-07-12 23:57:01
1              2             357201  2015-06-22 23:58:31  2016-07-12 23:57:01
2              3             356755  2015-06-22 23:58:31  2016-07-12 23:57:01
3              4             357213  2015-06-22 23:58:31  2016-07-12 23:57:01
4              5             357204  2015-06-22 23:58:31  2016-07-12 23:57:01
..           ...                ...                  ...                  ...
141          142             352730  2015-06-22 23:58:31  2016-07-12 23:57:01
142          143             352729  2015-06-22 23:58:31  2016-07-12 23:57:01
143          144             352730  2015-06-22 23:58:31  2016-07-12 23:57:01
144          145             352730  2015-06-22 23:58:31  2016-07-12 23:57:01
145          146             147666  2015-06-22 23:58:31  2015-12-01 04:21:01

[146 rows x 4 columns]


## 2.3 Moving Window

In [14]:
df = pd.read_sql("""
SELECT detector_id, count, LAG(count) OVER (PARTITION BY detector_id ORDER BY starttime) AS previous_count
FROM readings
WHERE detector_id = 146
LIMIT 500;
""", db_connection)
print(df)

# save to pickle
df.to_pickle("proj6_ex03_detector_146_lag.pkl")

     detector_id  count  previous_count
0            146      1             NaN
1            146      0             1.0
2            146      0             0.0
3            146      0             0.0
4            146      0             0.0
..           ...    ...             ...
495          146      0             0.0
496          146      2             0.0
497          146      2             2.0
498          146      2             2.0
499          146      2             2.0

[500 rows x 3 columns]


## 2.4 Window

In [15]:
df = pd.read_sql("""
SELECT detector_id, count, SUM(count) OVER (PARTITION BY detector_id ORDER BY starttime ROWS BETWEEN CURRENT ROW AND 10 FOLLOWING) AS window_sum
FROM readings
WHERE detector_id = 146
LIMIT 500;
""", db_connection)
print(df)

# save to pickle
df.to_pickle("proj6_ex04_detector_146_sum.pkl")

     detector_id  count  window_sum
0            146      1           2
1            146      0           2
2            146      0           2
3            146      0           2
4            146      0           2
..           ...    ...         ...
495          146      0          20
496          146      2          22
497          146      2          21
498          146      2          21
499          146      2          21

[500 rows x 3 columns]
